In [1]:
import pandas as pd
import numpy as np

In [2]:
scenario = 'real'

In [3]:
top_down = range(1024)
bottom_up = range(1024, 2048)

In [4]:
data_train = pd.read_csv('./data/{}_q_types_features_train_b.csv'.format(scenario), header=None)
#data_train['_scen'] = 'train'
data_val = pd.read_csv('./data/{}_q_types_features_val_b.csv'.format(scenario), header=None)
#data_val['_scen'] = 'val'
data = pd.concat([data_train, data_val])
del data_train
del data_val

In [5]:
data[2048] = data[2048].astype(int)
data.sort_values(by=2048, inplace=True)

In [6]:
data.shape

(658111, 2050)

In [7]:
details_train = pd.read_csv('./data/predictions_details_{}_train'.format(scenario), header=None)
details_val = pd.read_csv('./data/predictions_details_{}_val'.format(scenario), header=None)
details = pd.concat([details_train, details_val])
details[2] = details[2].astype(int)
details.sort_values(by=2, inplace=True)

predicted = details[0].values
target = details[1].values

data['my_predicted'] = predicted
data['my_target'] = target
data['score'] = details[3].values

In [8]:
(data[2048] != details[2].values).astype(int).sum()

0

In [9]:
details_train = pd.read_csv('./data/predictions_details_base_{}_train'.format(scenario), header=None)
details_val = pd.read_csv('./data/predictions_details_base_{}_val'.format(scenario), header=None)
details = pd.concat([details_train, details_val])
details[2] = details[2].astype(int)
details.sort_values(by=2, inplace=True)

predicted = details[0].values
target = details[1].values
data['base_predicted'] = predicted
data['base_target'] = target
data['base_score'] = details[3].values

In [10]:
(data[2048] != details[2].values).astype(int).sum()

0

In [11]:
#labels_train = pd.read_csv('./data/predictions_{}_train'.format(scenario), header=None).values[:,0]
#labels_val = pd.read_csv('./data/predictions_{}_val'.format(scenario), header=None).values[:,0]
#labels = labels_train.tolist() + labels_val.tolist()
#data['my_acc'] = labels
#assert labels_train.shape[0] + labels_val.shape[0] == len(labels)

In [12]:
#data.to_csv('./concatenated_data/{}.csv'.format(scenario), index=False)

In [13]:
data[:3]

,0,1,2,3,4,5,6,7,8,9,...,2046,2047,2048,2049,my_predicted,my_target,score,base_predicted,base_target,base_score
0,3.621112,0.0,0.362051,0.0,0.0,0.0,0.0,0.0,0.0,3.502063,...,0.000000,2.856147,9000,how many,841,841,1.0,2195,841,0.0
1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,7.530718,0.654848,9001,what color are the,1204,2542,0.0,759,2542,0.0
2,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,1.289435,0.000000,9002,what is the,2529,2529,1.0,2529,2529,1.0


In [14]:
data['sum_top_down'] = data[top_down].sum(1)
data['sum_bottom_up'] = data[bottom_up].sum(1)

In [15]:
data[:3]

,0,1,2,3,4,5,6,7,8,9,...,2048,2049,my_predicted,my_target,score,base_predicted,base_target,base_score,sum_top_down,sum_bottom_up
0,3.621112,0.0,0.362051,0.0,0.0,0.0,0.0,0.0,0.0,3.502063,...,9000,how many,841,841,1.0,2195,841,0.0,515.023036,555.346463
1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,9001,what color are the,1204,2542,0.0,759,2542,0.0,735.734441,733.956401
2,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,9002,what is the,2529,2529,1.0,2529,2529,1.0,946.130073,1108.889205


In [16]:
data.columns.tolist()[-10:]

[2048,
 2049,
 'my_predicted',
 'my_target',
 'score',
 'base_predicted',
 'base_target',
 'base_score',
 'sum_top_down',
 'sum_bottom_up']

In [17]:
data[data.columns.tolist()[-10:]].to_csv('./concatenated_data/{}_prediction_b.csv'.format(scenario), index=False)

In [18]:
data[:5][data.columns.tolist()[-10:]]

,2048,2049,my_predicted,my_target,score,base_predicted,base_target,base_score,sum_top_down,sum_bottom_up
0,9000,how many,841,841,1.0,2195,841,0.0,515.023036,555.346463
1,9001,what color are the,1204,2542,0.0,759,2542,0.0,735.734441,733.956401
2,9002,what is the,2529,2529,1.0,2529,2529,1.0,946.130073,1108.889205
3,25000,what is,1539,1539,1.0,1539,1539,1.0,832.034498,1070.618597
4,25001,what,991,2681,0.0,2681,2681,1.0,437.906643,658.692889


In [19]:
(data[2049]>0).sum()

658111

In [20]:
data.shape

(658111, 2058)

In [21]:
# column 2048 -> query type
_sum = data.groupby(2049).sum().round(1)
count = data.groupby(2049).count()

activations_sum = pd.concat([pd.DataFrame(_sum[top_down].sum(1), columns=['sum_top_down']), 
                           pd.DataFrame(_sum[bottom_up].sum(1), columns=['sum_bottom_up'])], 1)
_sum = (activations_sum.sum_top_down + activations_sum.sum_bottom_up).values
activations_sum = activations_sum/np.array([_sum, _sum]).T

activations_count = pd.concat([pd.DataFrame(count[top_down].sum(1), columns=['count_top_down']), 
                             pd.DataFrame(count[bottom_up].sum(1), columns=['count_bottom_up'])], 1)
_sum = (activations_count.count_top_down + activations_count.count_bottom_up).values
activations_count = activations_count/np.array([_sum, _sum]).T

summary = pd.concat([activations_sum, activations_count], 1)

In [22]:
summary

,sum_top_down,sum_bottom_up,count_top_down,count_bottom_up
2049,,,,
"""take me",0.492939,0.507061,0.5,0.5
1. about,0.493725,0.506275,0.5,0.5
1. approximately,0.505704,0.494296,0.5,0.5
1. are,0.475826,0.524174,0.5,0.5
1. do,0.518744,0.481256,0.5,0.5
1. does,0.418094,0.581906,0.5,0.5
1. what,0.511447,0.488553,0.5,0.5
3 people,0.491350,0.508650,0.5,0.5
3 poles,0.503313,0.496687,0.5,0.5


In [23]:
#p = 1.5 #90%

mean_dif = (summary.sum_top_down - summary.sum_bottom_up).mean()
std_dif = (summary.sum_top_down - summary.sum_bottom_up).std()
data_dif = (summary.sum_top_down - summary.sum_bottom_up)
#data_diff = pd.DataFrame( (data_dif < mean_dif - p * std_dif).astype(int) + 
#                         (data_dif > mean_dif + p * std_dif).astype(int) > 0 )

significantly_different = summary.copy()#.loc[data_diff[data_diff[0]].index]
significantly_different['diff'] = data_dif.round(2)
abstract_rep_count = pd.DataFrame(data[2049].value_counts())
abstract_representativity = np.round(abstract_rep_count/data.shape[0] *100, 4)
print(abstract_representativity.shape)
significantly_different['_count'] = abstract_rep_count.loc[significantly_different.index]
#significantly_different['represents %'] = abstract_representativity.loc[significantly_different.index]
significantly_different['over_median'] = abstract_representativity.loc[significantly_different.index] > float(abstract_representativity.median())
print(float(abstract_representativity.median()))
significantly_different['winner'] = np.array(['top_down',
                                              'bottom_up'])[(significantly_different.sum_bottom_up 
                                                            > significantly_different.sum_top_down).astype(int)]

#np.round(float(data_dif[data_diff[0]].shape[0]) / data_diff.shape[0] * 100, 2)

(1650, 1)
0.0003


In [24]:
significantly_different.shape

(1650, 8)

In [25]:
significantly_different.to_csv('./concatenated_data/queries_summary_{}_b.csv'.format(scenario), index=True)

In [26]:
significantly_different#[significantly_different._count>1].sort_values(by='winner')

,sum_top_down,sum_bottom_up,count_top_down,count_bottom_up,diff,_count,over_median,winner
2049,,,,,,,,
"""take me",0.492939,0.507061,0.5,0.5,-0.01,1,False,bottom_up
1. about,0.493725,0.506275,0.5,0.5,-0.01,1,False,bottom_up
1. approximately,0.505704,0.494296,0.5,0.5,0.01,2,False,top_down
1. are,0.475826,0.524174,0.5,0.5,-0.05,2,False,bottom_up
1. do,0.518744,0.481256,0.5,0.5,0.04,1,False,top_down
1. does,0.418094,0.581906,0.5,0.5,-0.16,2,False,bottom_up
1. what,0.511447,0.488553,0.5,0.5,0.02,2,False,top_down
3 people,0.491350,0.508650,0.5,0.5,-0.02,1,False,bottom_up
3 poles,0.503313,0.496687,0.5,0.5,0.01,1,False,top_down


In [27]:
2+2

4